In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from datetime import datetime, timedelta
import calendar
import threading
import time


class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Task Burst")  # Changed app name to Task Burst
        self.root.geometry("600x800")
        self.root.configure(bg="#f0f8ff")  # Light Blue Background

        # Task/Event List
        self.tasks = []
        self.birthdays = []
        self.meetings = []

        # Title Label
        tk.Label(self.root, text="Task Burst", font=("Helvetica", 20, "bold"), bg="#4682b4", fg="white", pady=10).pack(fill="x")

        # Notebook (Tabbed View)
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill="both")

        # Tabs
        self.create_task_tab()
        self.create_birthdays_tab()
        self.create_meetings_tab()
        self.create_calendar_tab()

        # Start the reminder checker thread
        self.reminder_thread = threading.Thread(target=self.check_reminders, daemon=True)
        self.reminder_thread.start()

    # Tab for To-Do Tasks
    def create_task_tab(self):
        self.task_tab = tk.Frame(self.notebook, bg="#f0f8ff")
        self.notebook.add(self.task_tab, text="Tasks")

        # Entry Frame for Tasks
        entry_frame = tk.Frame(self.task_tab, bg="#f0f8ff")
        entry_frame.pack(fill="x", pady=10)

        tk.Label(entry_frame, text="Task Title:", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.task_input = tk.Entry(entry_frame, font=("Helvetica", 12), width=30)
        self.task_input.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(entry_frame, text="Due Date (YYYY-MM-DD):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=1, column=0, padx=5, pady=5, sticky="e")
        self.task_date_input = tk.Entry(entry_frame, font=("Helvetica", 12), width=30)
        self.task_date_input.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(entry_frame, text="Due Time (HH:MM 24hr):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=2, column=0, padx=5, pady=5, sticky="e")
        self.task_time_input = tk.Entry(entry_frame, font=("Helvetica", 12), width=30)
        self.task_time_input.grid(row=2, column=1, padx=5, pady=5)

        tk.Button(entry_frame, text="Add Task", bg="#32cd32", fg="white", font=("Helvetica", 12, "bold"), command=self.add_task).grid(row=3, columnspan=2, pady=10)

        # Task List Frame
        self.task_frame = tk.Frame(self.task_tab, bg="#f0f8ff")
        self.task_frame.pack(pady=10, fill="both", expand=True)

    # Method to Add Task
    def add_task(self):
        task_title = self.task_input.get().strip()
        task_date = self.task_date_input.get().strip()
        task_time = self.task_time_input.get().strip()

        if task_title and task_date and task_time:
            try:
                due_datetime = datetime.strptime(f"{task_date} {task_time}", "%Y-%m-%d %H:%M")
                task_label = tk.Label(self.task_frame, text=f"{task_title} (Due: {due_datetime.strftime('%b %d, %Y %I:%M %p')})",
                                      bg="#add8e6", fg="black", font=("Helvetica", 14), pady=5, padx=10, anchor="w")
                task_label.pack(fill="x", pady=2)
                self.tasks.append({"label": task_label, "due_datetime": due_datetime, "completed": False})
                self.task_input.delete(0, tk.END)
                self.task_date_input.delete(0, tk.END)
                self.task_time_input.delete(0, tk.END)
            except ValueError:
                messagebox.showerror("Input Error", "Invalid date or time format. Please use YYYY-MM-DD for date and HH:MM (24hr) for time.")
        else:
            messagebox.showwarning("Input Error", "All fields are required!")

    # Birthday Tab with Reminder
    def create_birthdays_tab(self):
        self.birthday_tab = tk.Frame(self.notebook, bg="#f0f8ff")
        self.notebook.add(self.birthday_tab, text="Birthdays")
        tk.Label(self.birthday_tab, text="Add Birthdays", bg="#4682b4", fg="white", font=("Helvetica", 14, "bold")).pack(fill="x")

        birthday_frame = tk.Frame(self.birthday_tab, bg="#f0f8ff")
        birthday_frame.pack(pady=10, fill="x")

        tk.Label(birthday_frame, text="Name:", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.birthday_name_input = tk.Entry(birthday_frame, font=("Helvetica", 12), width=30)
        self.birthday_name_input.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(birthday_frame, text="Birthday Date (MM-DD):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=1, column=0, padx=5, pady=5, sticky="e")
        self.birthday_date_input = tk.Entry(birthday_frame, font=("Helvetica", 12), width=30)
        self.birthday_date_input.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(birthday_frame, text="Reminder Time (HH:MM 24hr):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=2, column=0, padx=5, pady=5, sticky="e")
        self.birthday_reminder_input = tk.Entry(birthday_frame, font=("Helvetica", 12), width=30)
        self.birthday_reminder_input.grid(row=2, column=1, padx=5, pady=5)

        tk.Button(birthday_frame, text="Add Birthday", bg="#32cd32", fg="white", font=("Helvetica", 12, "bold"), command=self.add_birthday).grid(row=3, columnspan=2, pady=10)

        self.birthday_list_frame = tk.Frame(self.birthday_tab, bg="#f0f8ff")
        self.birthday_list_frame.pack(pady=10, fill="both", expand=True)

    # Method to Add Birthday with Reminder
    def add_birthday(self):
        name = self.birthday_name_input.get().strip()
        date = self.birthday_date_input.get().strip()
        reminder_time = self.birthday_reminder_input.get().strip()

        if name and date and reminder_time:
            try:
                birthday_datetime = datetime.strptime(f"{date} {reminder_time}", "%m-%d %H:%M")
                birthday_label = tk.Label(self.birthday_list_frame, text=f"{name} (Birthday: {birthday_datetime.strftime('%b %d, %Y %I:%M %p')})",
                                          bg="#f0e68c", fg="black", font=("Helvetica", 14), pady=5, padx=10, anchor="w")
                birthday_label.pack(fill="x", pady=2)
                self.birthdays.append({"label": birthday_label, "birthday_datetime": birthday_datetime, "completed": False})
                self.birthday_name_input.delete(0, tk.END)
                self.birthday_date_input.delete(0, tk.END)
                self.birthday_reminder_input.delete(0, tk.END)
            except ValueError:
                messagebox.showerror("Input Error", "Invalid date or time format. Please use MM-DD for date and HH:MM (24hr) for reminder time.")
        else:
            messagebox.showwarning("Input Error", "All fields are required!")

    # Meeting Tab with Date and Time
    def create_meetings_tab(self):
        self.meeting_tab = tk.Frame(self.notebook, bg="#f0f8ff")
        self.notebook.add(self.meeting_tab, text="Meetings")
        tk.Label(self.meeting_tab, text="Add Meetings", bg="#4682b4", fg="white", font=("Helvetica", 14, "bold")).pack(fill="x")

        meeting_frame = tk.Frame(self.meeting_tab, bg="#f0f8ff")
        meeting_frame.pack(pady=10, fill="x")

        tk.Label(meeting_frame, text="Meeting Title:", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.meeting_title_input = tk.Entry(meeting_frame, font=("Helvetica", 12), width=30)
        self.meeting_title_input.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(meeting_frame, text="Meeting Date (YYYY-MM-DD):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=1, column=0, padx=5, pady=5, sticky="e")
        self.meeting_date_input = tk.Entry(meeting_frame, font=("Helvetica", 12), width=30)
        self.meeting_date_input.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(meeting_frame, text="Meeting Time (HH:MM 24hr):", bg="#f0f8ff", font=("Helvetica", 12)).grid(row=2, column=0, padx=5, pady=5, sticky="e")
        self.meeting_time_input = tk.Entry(meeting_frame, font=("Helvetica", 12), width=30)
        self.meeting_time_input.grid(row=2, column=1, padx=5, pady=5)

        tk.Button(meeting_frame, text="Add Meeting", bg="#32cd32", fg="white", font=("Helvetica", 12, "bold"), command=self.add_meeting).grid(row=3, columnspan=2, pady=10)

        self.meeting_list_frame = tk.Frame(self.meeting_tab, bg="#f0f8ff")
        self.meeting_list_frame.pack(pady=10, fill="both", expand=True)

    # Method to Add Meeting
    def add_meeting(self):
        title = self.meeting_title_input.get().strip()
        date = self.meeting_date_input.get().strip()
        time = self.meeting_time_input.get().strip()

        if title and date and time:
            try:
                meeting_datetime = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
                meeting_label = tk.Label(self.meeting_list_frame, text=f"{title} (Meeting on: {meeting_datetime.strftime('%b %d, %Y %I:%M %p')})",
                                         bg="#add8e6", fg="black", font=("Helvetica", 14), pady=5, padx=10, anchor="w")
                meeting_label.pack(fill="x", pady=2)
                self.meetings.append({"label": meeting_label, "meeting_datetime": meeting_datetime})
                self.meeting_title_input.delete(0, tk.END)
                self.meeting_date_input.delete(0, tk.END)
                self.meeting_time_input.delete(0, tk.END)
            except ValueError:
                messagebox.showerror("Input Error", "Invalid date or time format. Please use YYYY-MM-DD for date and HH:MM (24hr) for time.")
        else:
            messagebox.showwarning("Input Error", "All fields are required!")

    # Calendar Tab
    def create_calendar_tab(self):
        self.calendar_tab = tk.Frame(self.notebook, bg="#f0f8ff")
        self.notebook.add(self.calendar_tab, text="Calendar")

        current_year = datetime.now().year
        calendar_text = calendar.TextCalendar(calendar.SUNDAY).formatyear(current_year)

        tk.Label(self.calendar_tab, text="Calendar for the Year", font=("Helvetica", 16, "bold"), bg="#4682b4", fg="white").pack(pady=10)
        tk.Label(self.calendar_tab, text=calendar_text, font=("Courier", 10), bg="#f0f8ff").pack()

    # Reminder Checker (for birthdays or meetings)
    def check_reminders(self):
        while True:
            current_time = datetime.now()
            for task in self.tasks:
                if not task["completed"] and task["due_datetime"] <= current_time:
                    print(f"Reminder: Task {task['label'].cget('text')} is due!")
            for birthday in self.birthdays:
                if not birthday["completed"] and birthday["birthday_datetime"] <= current_time:
                    print(f"Reminder: Birthday {birthday['label'].cget('text')} is today!")
            for meeting in self.meetings:
                if meeting["meeting_datetime"] <= current_time:
                    print(f"Reminder: Meeting {meeting['label'].cget('text')} is happening!")
            time.sleep(60)  # Check reminders every 60 seconds


if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()
